### Introduction
##### This notebook has all the code you need to create your own chatbot with custom knowledge base using GPT-3.

#### Download the data for your custom knowledge base
##### For the demonstration purposes we are going to use ----- as our knowledge base. You can download them to your local folder from the github repository by running the code below. Alternatively, you can put your own custom data into the local folder.

In [1]:
! git clone https://github.com/irina1nik/context_data.git
    

Cloning into 'context_data'...


#### Install the dependicies
##### Run the code below to install the depencies we need for our functions

In [ ]:
!pip install llama-index==0.5.6
!pip install langchain==0.0.148

#### Define the functions
##### The following code defines the functions we need to construct the index and query it

In [13]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTVectorStoreIndex, LLMPredictor, PromptHelper,load_index_from_storage,StorageContext, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    index.storage_context.persist(persist_dir="index.json")

    return index

def ask_ai():
    storage_context = StorageContext.from_defaults(persist_dir="index.json")
    index = load_index_from_storage(storage_context)
    while True: 
        query = input("What do you want to ask? ")
        query_engine = index.as_query_engine()
        response = query_engine.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

#### Set OpenAI API Key
##### You need an OPENAI API key to be able to run this code.

In [ ]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

#### Construct an index
##### Now we are ready to construct the index. This will take every file in the folder 'data', split it into chunks, and embed it with OpenAI's embeddings API.

In [5]:
construct_index("context_data/data")


#### Ask questions
###### It's time to have fun and test our AI. Run the function that queries GPT and type your question into the input.

##### If you've used the provided example data for your custom knowledge base, here are a few questions that you can ask:

##### Why people cook at home? Make classification
##### Make classification about what frustrates people about cooking?
##### Brainstorm marketing campaign ideas for an air fryer that would appeal people that cook at home
##### Which kitchen appliences people use most often?
##### What people like about cooking at home?

In [ ]:
ask_ai()

What do you want to ask? what people like about cooking at home


Response: <b>
People like cooking at home because it allows them to relax and unwind after a long day, be in control of what goes into their meals, customize their meals to their family's preferences, spend time with their family in the kitchen, experiment with different recipes and ingredients, and make healthier and more affordable meals than eating out all the time.</b>